In [1]:
surname = 'Бабижаев'

alp = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
w = [1, 42, 21, 21, 34,  6, 44, 26, 18, 44, 38, 26, 14, 43,  4, 49, 45,
     7, 42, 29,  4,  9, 36, 34, 31, 29,  5, 30,  4, 19, 28, 25, 33]

d = dict(zip(alp, w))
variant = sum([d[el] for el in surname.lower()]) % 40 + 1

print('Задача № 1: ', variant % 3 + 1)
print('Задача № 2: ', variant % 2 + 1)

Задача № 1:  1
Задача № 2:  1


In [2]:
from pyspark.sql import SparkSession
import os
import math
from pyspark.sql.functions import explode, split, col, avg, count, lit, pow, sum, collect_list, struct, when, expr, sqrt
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [3]:
os.environ['JAVA_HOME'] = '/opt/homebrew/opt/openjdk@17/libexec/openjdk.jdk/Contents/Home'
os.environ['PATH'] = f'{os.environ['JAVA_HOME']}/bin:{os.environ['PATH']}'
os.environ['_JAVA_OPTIONS'] = '-Dhadoop.security.authentication=simple -Duser.name=spark'


spark = SparkSession.builder.appName('hw4').getOrCreate()

Picked up _JAVA_OPTIONS: -Dhadoop.security.authentication=simple -Duser.name=spark
Picked up _JAVA_OPTIONS: -Dhadoop.security.authentication=simple -Duser.name=spark
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/01 02:25:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/01 02:25:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/12/01 02:25:34 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## Задание 1

In [4]:
df = spark.read \
    .option('header', 'true') \
    .csv('movies.csv')

print(df.columns)

['movieId', 'title', 'genres']


In [5]:
SELECTED_GENRES = ['Animation', 'Romance', 'Documentary']

genres_df = df.withColumn('genre', explode(split(col('genres'), '\\|'))) \
    .select('movieId', 'title', 'genre').filter(col('genre').isin(SELECTED_GENRES))

genres_df.show(10)

+-------+--------------------+---------+
|movieId|               title|    genre|
+-------+--------------------+---------+
|      1|    Toy Story (1995)|Animation|
|      3|Grumpier Old Men ...|  Romance|
|      4|Waiting to Exhale...|  Romance|
|      7|      Sabrina (1995)|  Romance|
|     11|American Presiden...|  Romance|
|     13|        Balto (1995)|Animation|
|     15|Cutthroat Island ...|  Romance|
|     17|Sense and Sensibi...|  Romance|
|     25|Leaving Las Vegas...|  Romance|
|     28|   Persuasion (1995)|  Romance|
+-------+--------------------+---------+
only showing top 10 rows



* Задание 1.1

In [6]:
genres_df.groupBy('genre').count().show(10)

+-----------+-----+
|      genre|count|
+-----------+-----+
|    Romance| 1596|
|Documentary|  440|
|  Animation|  611|
+-----------+-----+



* Задание 1.2

In [7]:
ratings_df = spark.read \
    .option('header', 'true') \
    .csv('ratings.csv')

ratings_df.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [8]:
print(50 * '-')
print('Animation')
ratings_df.join(genres_df, ratings_df.movieId == genres_df.movieId).select('userId',
                                                                           'title', 'genre', col('rating').cast('double')).where(col('genre') == 'Animation').groupBy('title', 'genre').count()\
    .orderBy(col('count').desc())\
    .show(10)

print(50 * '-')
print('Documentary')
ratings_df.join(genres_df, ratings_df.movieId == genres_df.movieId).select('userId',
                                                                           'title', 'genre', col('rating').cast('double')).where(col('genre') == 'Documentary').groupBy('title', 'genre').count()\
    .orderBy(col('count').desc())\
    .show(10)
print(50 * '-')
print('Romance')
ratings_df.join(genres_df, ratings_df.movieId == genres_df.movieId).select('userId',
                                                                           'title', 'genre', col('rating').cast('double')).where(col('genre') == 'Romance').groupBy('title', 'genre').count()\
    .orderBy(col('count').desc())\
    .show(10)

--------------------------------------------------
Animation
+--------------------+---------+-----+
|               title|    genre|count|
+--------------------+---------+-----+
|    Toy Story (1995)|Animation|  215|
|      Aladdin (1992)|Animation|  183|
|Lion King, The (1...|Animation|  172|
|        Shrek (2001)|Animation|  170|
|Beauty and the Be...|Animation|  146|
| Finding Nemo (2003)|Animation|  141|
|Monsters, Inc. (2...|Animation|  132|
|Incredibles, The ...|Animation|  125|
|           Up (2009)|Animation|  105|
|       WALL·E (2008)|Animation|  104|
+--------------------+---------+-----+
only showing top 10 rows

--------------------------------------------------
Documentary
+--------------------+-----------+-----+
|               title|      genre|count|
+--------------------+-----------+-----+
|Bowling for Colum...|Documentary|   58|
|Super Size Me (2004)|Documentary|   50|
|Fahrenheit 9/11 (...|Documentary|   37|
|   Roger & Me (1989)|Documentary|   31|
|  Hoop Dreams (1

* Задание 1.3

In [9]:
print(50 * '-')
print('Animation')
ratings_df.join(genres_df, ratings_df.movieId == genres_df.movieId).select('userId',
                                                                           'title', 'genre', col('rating').cast('double')).where(col('genre') == 'Animation').groupBy('title', 'genre').count().where(col('count') > 10)\
    .orderBy(col('count'))\
    .show(10)

print(50 * '-')
print('Documentary')
ratings_df.join(genres_df, ratings_df.movieId == genres_df.movieId).select('userId',
                                                                           'title', 'genre', col('rating').cast('double')).where(col('genre') == 'Documentary').groupBy('title', 'genre').count().where(col('count') > 10)\
    .orderBy(col('count'))\
    .show(10)
print(50 * '-')
print('Romance')
ratings_df.join(genres_df, ratings_df.movieId == genres_df.movieId).select('userId',
                                                                           'title', 'genre', col('rating').cast('double')).where(col('genre') == 'Romance').groupBy('title', 'genre').count().where(col('count') > 10)\
    .orderBy(col('count'))\
    .show(10)

--------------------------------------------------
Animation
+--------------------+---------+-----+
|               title|    genre|count|
+--------------------+---------+-----+
|Oliver & Company ...|Animation|   11|
|How the Grinch St...|Animation|   11|
|Ponyo (Gake no ue...|Animation|   11|
|All Dogs Go to He...|Animation|   11|
|   Happy Feet (2006)|Animation|   11|
|   Persepolis (2007)|Animation|   11|
|Polar Express, Th...|Animation|   11|
|Meet the Robinson...|Animation|   11|
|Hotel Transylvani...|Animation|   11|
|Rise of the Guard...|Animation|   12|
+--------------------+---------+-----+
only showing top 10 rows

--------------------------------------------------
Documentary
+--------------------+-----------+-----+
|               title|      genre|count|
+--------------------+-----------+-----+
|Thin Blue Line, T...|Documentary|   11|
|Koyaanisqatsi (a....|Documentary|   11|
|   Inside Job (2010)|Documentary|   12|
|King of Kong, The...|Documentary|   12|
|Exit Through the

* Задание 1.4

In [10]:
print(50 * '-')
print('Animation')
ratings_df.join(genres_df, 'movieId') \
    .select('title', 'genre', 'rating') \
    .where(col('genre') == 'Animation') \
    .groupBy('title', 'genre') \
    .agg(
        avg('rating').alias('avg_rating'),
        count('*').alias('rating_count')
) \
    .filter(col('rating_count') >= 10) \
    .orderBy(col('avg_rating').desc()) \
    .select('title', 'avg_rating', 'rating_count') \
    .show(10)

print(50 * '-')
print('Documentary')
ratings_df.join(genres_df, 'movieId') \
    .select('title', 'genre', 'rating') \
    .where(col('genre') == 'Documentary') \
    .groupBy('title', 'genre') \
    .agg(
        avg('rating').alias('avg_rating'),
        count('*').alias('rating_count')
) \
    .filter(col('rating_count') >= 10) \
    .orderBy(col('avg_rating').desc()) \
    .select('title', 'avg_rating', 'rating_count') \
    .show(10)

print(50 * '-')
print('Romance')
ratings_df.join(genres_df, 'movieId') \
    .select('title', 'genre', 'rating') \
    .where(col('genre') == 'Romance') \
    .groupBy('title', 'genre') \
    .agg(
        avg('rating').alias('avg_rating'),
        count('*').alias('rating_count')
) \
    .filter(col('rating_count') >= 10) \
    .orderBy(col('avg_rating').desc()) \
    .select('title', 'avg_rating', 'rating_count') \
    .show(10)

--------------------------------------------------
Animation
+--------------------+-----------------+------------+
|               title|       avg_rating|rating_count|
+--------------------+-----------------+------------+
|Creature Comforts...|             4.25|          12|
| Mary and Max (2009)|              4.2|          10|
|   Persepolis (2007)|4.181818181818182|          11|
|Grave of the Fire...|          4.15625|          16|
|Spirited Away (Se...|4.155172413793103|          87|
|Ghost in the Shel...|4.148148148148148|          27|
|Batman: Mask of t...|4.115384615384615|          13|
|  Toy Story 3 (2010)|4.109090909090909|          55|
|Girl Who Leapt Th...|              4.1|          10|
|Wallace & Gromit:...|4.092592592592593|          27|
+--------------------+-----------------+------------+
only showing top 10 rows

--------------------------------------------------
Documentary
+--------------------+------------------+------------+
|               title|        avg_ratin

* Задача 1.5

In [11]:
print(50 * '-')
print('Animation')
ratings_df.join(genres_df, 'movieId') \
    .select('title', 'genre', 'rating') \
    .where(col('genre') == 'Animation') \
    .groupBy('title', 'genre') \
    .agg(
        avg('rating').alias('avg_rating'),
        count('*').alias('rating_count')
) \
    .filter(col('rating_count') >= 10) \
    .orderBy(col('avg_rating')) \
    .select('title', 'avg_rating', 'rating_count') \
    .show(10)

print(50 * '-')
print('Documentary')
ratings_df.join(genres_df, 'movieId') \
    .select('title', 'genre', 'rating') \
    .where(col('genre') == 'Documentary') \
    .groupBy('title', 'genre') \
    .agg(
        avg('rating').alias('avg_rating'),
        count('*').alias('rating_count')
) \
    .filter(col('rating_count') >= 10) \
    .orderBy(col('avg_rating')) \
    .select('title', 'avg_rating', 'rating_count') \
    .show(10)

print(50 * '-')
print('Romance')
ratings_df.join(genres_df, 'movieId') \
    .select('title', 'genre', 'rating') \
    .where(col('genre') == 'Romance') \
    .groupBy('title', 'genre') \
    .agg(
        avg('rating').alias('avg_rating'),
        count('*').alias('rating_count')
) \
    .filter(col('rating_count') >= 10) \
    .orderBy(col('avg_rating')) \
    .select('title', 'avg_rating', 'rating_count') \
    .show(10)

--------------------------------------------------
Animation
+--------------------+------------------+------------+
|               title|        avg_rating|rating_count|
+--------------------+------------------+------------+
|   Shark Tale (2004)|2.3461538461538463|          13|
|Ice Age: Dawn of ...| 2.607142857142857|          14|
|   Happy Feet (2006)|2.6818181818181817|          11|
|American Tail: Fi...|               2.7|          10|
|All Dogs Go to He...|               2.7|          15|
|    Space Jam (1996)| 2.707547169811321|          53|
|Pete's Dragon (1977)|2.7666666666666666|          15|
|Small Soldiers (1...|2.8333333333333335|          18|
|Beavis and Butt-H...| 2.935483870967742|          31|
|Goofy Movie, A (1...|               3.0|          17|
+--------------------+------------------+------------+
only showing top 10 rows

--------------------------------------------------
Documentary
+--------------------+------------------+------------+
|               title|   

## Задание 2

In [12]:
rating_df = spark.read \
    .option('header', 'true') \
    .csv('ratings.csv')

rating_df.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



* Задание 2.1

In [13]:
df = ratings_df.withColumn('userId', col('userId').cast('integer')) \
    .withColumn('movieId', col('movieId').cast('integer')) \
    .withColumn('rating', col('rating').cast('double'))

train_init, test = df.randomSplit([0.8, 0.2], seed=42)

mean_rating = train_init.select(avg('rating')).collect()[0][0]
print(
    f'Средний рейтинг {mean_rating}')

test_with_predictions = test.withColumn('prediction', lit(mean_rating))

test_with_errors = test_with_predictions.withColumn(
    'squared_error', pow(col('rating') - col('prediction'), 2)
)

mse = test_with_errors.select(avg('squared_error')).collect()[0][0]
rmse = math.sqrt(mse)

print(f'RMSE: {rmse}')

Средний рейтинг 3.5039340762987417
RMSE: 1.0504377707169823


* Задание 2.2

In [ ]:
def user_based_collaborative_filtering(train_df, test_df, k_neighbors=10):

    user_means = train_df.groupBy('userId').agg(
        avg('rating').alias('user_mean'))

    train_normalized = train_df.join(user_means, 'userId') \
        .withColumn('normalized_rating', col('rating') - col('user_mean')) \
        .select('userId', 'movieId', 'normalized_rating')

    user_pairs = train_normalized.alias('a') \
        .join(train_normalized.alias('b'),
              (col('a.movieId') == col('b.movieId')) & (col('a.userId') < col('b.userId'))) \
        .select(
            col('a.userId').alias('user1'),
            col('b.userId').alias('user2'),
            col('a.normalized_rating').alias('rating1'),
            col('b.normalized_rating').alias('rating2')
    )

    similarities = user_pairs.groupBy('user1', 'user2') \
        .agg(
            sum(col('rating1') * col('rating2')).alias('dot_product'),
            sum(col('rating1') * col('rating1')).alias('norm1_sq'),
            sum(col('rating2') * col('rating2')).alias('norm2_sq'),
            count('*').alias('common_movies')
    ) \
        .filter(col('common_movies') >= 5) \
        .withColumn('similarity',
                    col('dot_product') / (sqrt(col('norm1_sq')) * sqrt(col('norm2_sq')))) \
        .select('user1', 'user2', 'similarity', 'common_movies')

    user_neighbors = similarities.groupBy('user1') \
        .agg(collect_list(struct('user2', 'similarity')).alias('neighbors')) \
        .withColumn('neighbors',
                    expr(f'slice(array_sort(neighbors, (a, b) -> case when b.similarity > a.similarity then 1 else -1 end), 1, {k_neighbors})'))

    neighbors_expanded = user_neighbors.select(
        col('user1').alias('userId'),
        explode('neighbors').alias('neighbor_info')
    ).select(
        'userId',
        col('neighbor_info.user2').alias('neighborId'),
        col('neighbor_info.similarity').alias('similarity')
    )

    predictions = test_df.alias('test') \
        .join(neighbors_expanded.alias('neighbors'), 'userId') \
        .join(train_df.alias('train'),
              (col('neighbors.neighborId') == col('train.userId')) &
              (col('test.movieId') == col('train.movieId'))) \
        .groupBy('test.userId', 'test.movieId') \
        .agg(
            sum(col('similarity') * col('train.rating')
                ).alias('weighted_sum'),
            sum(col('similarity')).alias('similarity_sum')
    ) \
        .withColumn('prediction',
                    when(col('similarity_sum') > 0, col(
                        'weighted_sum') / col('similarity_sum'))
                    .otherwise(None)) \
        .select('test.userId', 'test.movieId', 'prediction')

    return predictions


def calculate_rmse(predictions_df, test_df):
    joined = test_df.join(predictions_df, ['userId', 'movieId'])

    joined_with_pred = joined.filter(col('prediction').isNotNull())

    mse = joined_with_pred.select(
        avg((col('rating') - col('prediction')) ** 2)
    ).collect()[0][0]

    rmse = math.sqrt(mse)
    return rmse,


cf_predictions = user_based_collaborative_filtering(
    train_init, test, k_neighbors=10)

cf_rmse = calculate_rmse(cf_predictions, test)
print(f'RMSE: {cf_rmse}')

RMSE: (1.2546562076332535,)


##  Задание 3

* Задание 3.1

In [15]:
als = ALS(
    userCol='userId',
    itemCol='movieId',
    ratingCol='rating',
    seed=42,
    coldStartStrategy='drop',
)

param_grid = ParamGridBuilder() \
    .addGrid(als.rank, [5, 10, 15]) \
    .addGrid(als.regParam, [0.001, 0.01, 0.1, 1, 10]) \
    .build()

evaluator = RegressionEvaluator(
    metricName='rmse',
    labelCol='rating',
    predictionCol='prediction'
)

crossval = CrossValidator(
    estimator=als,
    estimatorParamMaps=param_grid,
    evaluator=evaluator,
    numFolds=4,
    seed=42
)

cv_model = crossval.fit(train_init)

best_model = cv_model.bestModel
best_rank = best_model.rank
best_reg_param = best_model._java_obj.parent().getRegParam()

print(f'Количество факторов (rank): {best_rank}')
print(f'Регуляризация (regParam): {best_reg_param}')

test_predictions = best_model.transform(test)
best_rmse = evaluator.evaluate(test_predictions)
print(f'RMSE : {best_rmse}')

25/12/01 02:25:42 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/12/01 02:25:42 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
25/12/01 02:25:42 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


Количество факторов (rank): 5
Регуляризация (regParam): 0.1
RMSE : 0.875577702656396


* Задание 3.2

In [17]:
print(
    f'Разница между коллаборативной фильтрацией и факторизацией матрицы: {cf_rmse[0] - best_rmse}')

Разница между коллаборативной фильтрацией и факторизацией матрицы: 0.37907850497685747
